Scraping Canada Postal Code

In [2]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

Scraping from Wikipedia. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood 

In [47]:
can_post = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header = 0)

can_post_df = can_post[0]

can_post_df.columns = ['PostalCode','Borough','Neighborhood']

can_post_df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [48]:
can_post_df.drop(can_post_df[can_post_df['Borough'] == 'Not assigned'].index, axis = 0, inplace = True)

can_post_df.reset_index(drop = True)

can_post_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [49]:
split_data = can_post_df["Neighborhood"].str.split("/")

df = pd.DataFrame(split_data)

df['liststring'] = df['Neighborhood'].apply(lambda x: ','.join(map(str, x)))

can_post_df['Neighborhood'] = df['liststring']

can_post_df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.


In [50]:
can_post_df[(can_post_df['Neighborhood'] =='Not assigned')&(can_post_df['Borough']!='Not assigned')]['Neighborhood'] =can_post_df[(can_post_df['Neighborhood'] =='Not assigned')&(can_post_df['Borough']!='Not assigned')]['Borough']

Check if there are any rows where Neighborhood is not assigned

In [51]:
len(can_post_df[can_post_df['Neighborhood'] =='Not assigned'])

0

In [52]:
can_post_df.shape

(103, 3)

In [179]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [14]:
postal_code = can_post_df['PostalCode'].iloc[0]

postal_code

'M3A'

In [15]:
import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print('latitude','longtitude')

KeyboardInterrupt: 

Geocode doesn't seem to be working, so importing from CSV file

In [53]:
geo_data = pd.read_csv('http://cocl.us/Geospatial_data',header = 0)

In [54]:
geo_data.columns = ['PostalCode','Latitude','Longitude']

geo_data

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Merging geodata with previous dataframe

In [78]:
test = can_post_df

can_post_df = can_post_df.merge(geo_data,'left',on='PostalCode')

can_post_df


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509
